In [22]:
%matplotlib inline
import os
import sys
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
import pickle as pk
from tqdm import tqdm
import pymaster as nmt
sys.path.append('../')
import matplotlib
import lenspyx
import camb
import emcee
import corner
from getdist import plots, MCSamples
from contextlib import contextmanager
from covariance import MeanAndCovariance
from likelihood import Delens_Theory
from likelihood import LH_simple2 as LH_simple
from likelihood import LH_HL2 as LH_HL

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

In [2]:
%load_ext autoreload
%autoreload 2
from delens import DelensAndCl

In [3]:
DC1 = DelensAndCl('delensing1.ini')

Simulation uses /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET1/Maps
Simulation uses /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET1/Maps
Delensing LiteBIRD-SET1 with LiteBIRD-SET1
Template is constructed using LiteBIRD-SET1
Delensing uses QE: p_eb
Reading coupling matrix from /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET1/Pseudo/workspace/coupling_matrix_Nside512_fsky_0p8_beam_None_Apo_C1_Deg_8.fits


In [4]:
DC2 = DelensAndCl('delensing2.ini')

Simulation uses /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET2/Maps
Simulation uses /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET3/Maps
Delensing LiteBIRD-SET2 with LiteBIRD-SET3
Template is constructed using LiteBIRD-SET2
Delensing uses QE: p_eb
Reading coupling matrix from /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET2/Pseudo/workspace/coupling_matrix_Nside512_fsky_0p8_beam_None_Apo_C1_Deg_8.fits


In [5]:
DC3 = DelensAndCl('delensing3.ini')

Simulation uses /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET3/Maps
Simulation uses /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET3/Maps
Delensing LiteBIRD-SET3 with LiteBIRD-SET3
Template is constructed using LiteBIRD-SET3
Delensing uses QE: p_eb
Reading coupling matrix from /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET3/Pseudo/workspace/coupling_matrix_Nside512_fsky_0p8_beam_None_Apo_C1_Deg_8.fits


In [6]:
camb_ini = '/global/cscratch1/sd/lonappan/S4BIRD/CAMB/BBSims_params.ini'

In [7]:
the = Delens_Theory(camb_ini,DC2.pseudocl_lib.b.lmax,DC1.delens_lib.get_N0(0),30,2.16)

In [8]:
lmin=20
lmax=180
m = MeanAndCovariance('Eff2',DC1,DC2,DC3,the,lmin,lmax)
lens_cov = m.lensed_cov_fid
del_cov = m.delensed_cov_fid

Covariance: 100%|██████████| 1000/1000 [00:00<00:00, 180206.40simulation/s]


In [9]:
LH_simple(lens_cov,3000,the,DC2.pseudocl_lib.b,lmin,lmax,'lensed').plot_posterior(m.get_biased('lensed').mean(axis=0))

../likelihood.py:135: RuntimeWarning: invalid value encountered in true_divide
  return (bb/self.dl)
../likelihood.py:128: RuntimeWarning: invalid value encountered in true_divide
  return self.cl_pp*(1 - (self.cl_pp/(self.cl_pp+self.N0)))
../likelihood.py:149: RuntimeWarning: divide by zero encountered in true_divide
  return (bb/self.dl)
../likelihood.py:142: RuntimeWarning: invalid value encountered in true_divide
  return (bb/self.dl)
100%|██████████| 3000/3000 [00:16<00:00, 179.64it/s]


r < 0.000436


In [10]:
LH_simple(del_cov,3000,the,DC2.pseudocl_lib.b,lmin,lmax,'delensed').plot_posterior(m.debiased_w_mc.mean(axis=0))

../likelihood.py:135: RuntimeWarning: invalid value encountered in true_divide
  return (bb/self.dl)
../likelihood.py:128: RuntimeWarning: invalid value encountered in true_divide
  return self.cl_pp*(1 - (self.cl_pp/(self.cl_pp+self.N0)))
../likelihood.py:149: RuntimeWarning: divide by zero encountered in true_divide
  return (bb/self.dl)
../likelihood.py:142: RuntimeWarning: invalid value encountered in true_divide
  return (bb/self.dl)
100%|██████████| 3000/3000 [00:16<00:00, 178.41it/s]

r < 0.000399


In [11]:
lens_c = m.get_unbiased('lensed').mean(axis=0)
del_c = m.get_unbiased('delensed').mean(axis=0)

In [12]:
LH_HL(lens_c,lens_cov,3000,the,DC2.pseudocl_lib.b,lmin,lmax,'lensed').plot_posterior(m.get_biased('lensed').mean(axis=0))

../likelihood.py:135: RuntimeWarning: invalid value encountered in true_divide
  return (bb/self.dl)
../likelihood.py:128: RuntimeWarning: invalid value encountered in true_divide
  return self.cl_pp*(1 - (self.cl_pp/(self.cl_pp+self.N0)))
../likelihood.py:149: RuntimeWarning: divide by zero encountered in true_divide
  return (bb/self.dl)
../likelihood.py:142: RuntimeWarning: invalid value encountered in true_divide
  return (bb/self.dl)
100%|██████████| 3000/3000 [00:21<00:00, 141.87it/s]

r < 0.000457


In [13]:
LH_HL(del_c,del_cov,3000,the,DC2.pseudocl_lib.b,lmin,lmax,'delensed').plot_posterior(m.debiased_w_mc.mean(axis=0))

../likelihood.py:135: RuntimeWarning: invalid value encountered in true_divide
  return (bb/self.dl)
../likelihood.py:128: RuntimeWarning: invalid value encountered in true_divide
  return self.cl_pp*(1 - (self.cl_pp/(self.cl_pp+self.N0)))
../likelihood.py:149: RuntimeWarning: divide by zero encountered in true_divide
  return (bb/self.dl)
../likelihood.py:142: RuntimeWarning: invalid value encountered in true_divide
  return (bb/self.dl)
100%|██████████| 3000/3000 [00:20<00:00, 146.25it/s]

r < 0.000377


In [14]:
np.mean(1-(m.debiased_w_mc.mean(axis=0)[m.select]/m.get_biased('lensed').mean(axis=0)[m.select]))

../likelihood.py:128: RuntimeWarning: invalid value encountered in true_divide
  return self.cl_pp*(1 - (self.cl_pp/(self.cl_pp+self.N0)))
../likelihood.py:149: RuntimeWarning: divide by zero encountered in true_divide
  return (bb/self.dl)
../likelihood.py:135: RuntimeWarning: invalid value encountered in true_divide
  return (bb/self.dl)


0.19749830497530912

In [15]:
1-377/457

0.17505470459518602